### Pre-processing the 19 Camden High Streets into Survival Format

In [1712]:
# import necessary libraries
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from requests import get
from urllib.parse import urlparse
from datetime import datetime

In [1713]:
%cd '/home/jovyan/work/Dissertation/Camden High Streets'

/home/jovyan/work/Dissertation/Camden High Streets


In [1714]:
os.getcwd()

'/home/jovyan/work/Dissertation/Camden High Streets'

### Read in the high street subsets

In [1715]:
Brecknock = pd.read_csv("Brecknock-points.csv", low_memory=False)

In [1716]:
Kentish = pd.read_csv("Kentish-points.csv", low_memory=False)

In [1717]:
Fortress = pd.read_csv("Fortress-points.csv", low_memory=False)

In [1718]:
CamdenT = pd.read_csv("CamdenT-points.csv", low_memory=False)

In [1719]:
Belsize = pd.read_csv("Belsize-points.csv", low_memory=False)

In [1720]:
Loudoun = pd.read_csv("Loudoun-points.csv", low_memory=False)

In [1721]:
Haverstock = pd.read_csv("Haverstock-points.csv", low_memory=False)

In [1722]:
SouthEnd = pd.read_csv("SouthEnd-points.csv", low_memory=False)

In [1723]:
#Highgate = pd.read_csv("Highgate-points.csv", low_memory=False)

In [1724]:
Cricklewood = pd.read_csv("Cricklewood-points.csv", low_memory=False)

In [1725]:
Regents = pd.read_csv("Regents-points.csv", low_memory=False)

In [1726]:
Queens = pd.read_csv("Queens-points.csv", low_memory=False)

In [1727]:
Kilburn = pd.read_csv("Kilburn-points.csv", low_memory=False)

In [1728]:
Mill = pd.read_csv("Mill-points.csv", low_memory=False)

In [1729]:
Fortune = pd.read_csv("Fortune-points.csv", low_memory=False)

In [1730]:
Willesden = pd.read_csv("Willesden-points.csv", low_memory=False)

In [1731]:
#Finchley = pd.read_csv("Finchley-points.csv", low_memory=False)

In [1732]:
Mansfield = pd.read_csv("Mansfield-points.csv", low_memory=False)

In [1733]:
WestEnd = pd.read_csv("WestEnd-points.csv", low_memory=False)

In [1734]:
Hampstead = pd.read_csv("Hampstead-points.csv", low_memory=False)

In [1735]:
Eversholt = pd.read_csv("Eversholt-points.csv", low_memory=False)

### Run the preprocessing template for each dataset 

#### 1. Brecknock 

In [1736]:
Brecknock = Brecknock.sort_values(by= ['status_date'])

In [1737]:
cols = ['V3', 'V4', 'V5', 'V13', 'V14', 'V15', 'V16', 'V17', 'V19', 'V20', 'V21', 'V26', 'V28', 'V30', 'V36', 'V38', 'V39', 'V48', 'V51', 'V54', 'V55', 'V56', 'V57', 
        'V58', 'V59', 'V60', 'V64', 'V70', 'V72', 'V77']

In [1738]:
#deleted those cols columns from the dataset as they only contain nulls

Brecknock = Brecknock.drop(cols, axis=1)

In [1739]:
Brecknock['new_date'] = pd.to_datetime(Brecknock['status_date'], errors='coerce')

In [1740]:
Brecknock = Brecknock.sort_values(by= ['new_date'])

In [1741]:
Brecknock['new_date'].loc[(Brecknock['new_date'] < '2010-04-01')] = '2010-04-01'

In [1742]:
Brecknock = Brecknock.sort_values(by= ['location_code', 'period'])

In [1743]:
Brecknock['locaton_code'] = Brecknock['location_code'].astype("category")

In [1744]:
locations = Brecknock['location_code'].unique()

In [1745]:
Brecknock.loc[Brecknock['name'].isnull(), 'name'] = Brecknock['location_code']

In [1746]:
Brecknock['status_date'] = pd.to_datetime(Brecknock['status_date'], errors='coerce')

In [1747]:
Brecknock['status_date'].iloc[Brecknock['status_date'] < '2010-04-01'] = '04/01/2010'

In [1748]:
Brecknock['status_date'] = pd.to_datetime(Brecknock['status_date'], errors='coerce')

In [1749]:
Brecknock['period2'] = pd.to_datetime(Brecknock['period'])

In [1750]:
Brecknock['period2'] = Brecknock['period2'] + pd.DateOffset(months=1)

In [1751]:
Brecknock['duration'] = Brecknock['period2'] - Brecknock['status_date']

In [1752]:
Brecknock['duration'] = Brecknock['duration'].dt.days

In [1753]:
Brecknock['study-start'] = '04/01/2010'

In [1754]:
Brecknock['study-start'] = pd.to_datetime(Brecknock['study-start'], errors='coerce')

In [1755]:
Brecknock['time-of-entry'] = Brecknock['status_date'] -  Brecknock['study-start']

In [1756]:
Brecknock['time-at-open'] = 0

In [1757]:
Brecknock['vacant'] = 1

In [1758]:
Brecknock = Brecknock.drop(Brecknock[(Brecknock['status'] == True)].index)

In [1759]:
Brecknock['location_code'] = Brecknock['location_code'].astype("category")

In [1760]:
locations = list(Brecknock['location_code'].unique())

In [1761]:
Brecknock = Brecknock.set_index('location_code')

In [1762]:
Brecknock['survival'] = 0

In [1763]:
Brecknock['survival'] = Brecknock['duration']

In [1764]:
Brecknock['entry'] = Brecknock['time-of-entry'].dt.days

In [1765]:
Brecknock['time-at-open'] = Brecknock['survival'] + Brecknock['entry']

In [1766]:
Brecknock['stat'] = Brecknock['status'].astype(int)

In [1767]:
Brecknock["StatusChange"] =  Brecknock['stat'].shift() != Brecknock['stat']

In [1768]:
Brecknock["StatusChange"].iloc[0] = False

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [1769]:
Brecknock["TenantChange"] =  Brecknock['name'].shift(-1) != Brecknock['name'] 

In [1770]:
Brecknock["TenantChange"].iloc[0] = False

In [1771]:
counter = 0 
stoppingCondition = len(locations)
Brecknock2 = pd.DataFrame(data=None, columns=Brecknock.columns, index=None)

while counter < stoppingCondition:
        i = locations[counter]
        m = Brecknock.loc[i, :].iloc[[ Brecknock.loc[i, :]['status_duration'].argmax()]]
        Brecknock2 = Brecknock2.append(m, ignore_index=False)
        counter += 1

In [1772]:
Brecknock3 = Brecknock2.drop_duplicates()

In [1773]:
Brecknock3.loc[Brecknock3['time-at-open'] == 4018, ['vacant']] = 0

In [1774]:
Brecknock3['highstreet'] = 'Brecknock'

In [1775]:
Brecknock3.to_csv('Brecknock-vacant-survival.csv')

#### 2. Kentish 

In [1776]:
Kentish = Kentish.sort_values(by= ['status_date'])

In [1777]:
cols = ['V3', 'V4', 'V5', 'V13', 'V14', 'V15', 'V16', 'V17', 'V19', 'V20', 'V21', 'V26', 'V28', 'V30', 'V36', 'V38', 'V39', 'V48', 'V51', 'V54', 'V55', 'V56', 'V57', 
        'V58', 'V59', 'V60', 'V64', 'V70', 'V72', 'V77']

In [1778]:
Kentish

,Unnamed: 0,Unnamed: 0.1,rates_code,rates_authority_id,rates_authority,region,location_code,address_no,address_street,address_town,...,V69,V70,V71,V72,V73,V74,V75,V76,V77,geometry
3880,329295,329295,51201460012,E09000007,Camden,Inner London - West,8079315000,GND - 2ND FLRS 146,ROYAL COLLEGE STREET,LONDON,...,D,NaN,E05000143,NaN,2013-07-01,0,CO,203.0,NaN,POINT (529304.7813367147 184070.2960136788)
4399,402250,402250,6.54002E+11,E09000007,Camden,Inner London - West,11659715000,UNIT 4 AT 21,BONNY STREET,LONDON,...,C,NaN,E05000130,NaN,NaN,0,CO,203.0,NaN,POINT (529149.7842568328 184183.290960255)
962,82795,82795,5.1202E+11,E09000007,Camden,Inner London - West,75247063,201,ROYAL COLLEGE STREET,LONDON,...,D,NaN,E05000130,NaN,2012-03-19,0,CS,249.0,NaN,POINT (529229.7802553794 184107.2885801889)
3481,123894,123894,0074200631009A,E09000007,Camden,Inner London - West,95659063,UNIT 7 AT 63,PRATT STREET,LONDON,...,L,NaN,E05000143,NaN,NaN,0,IF3,96.0,NaN,POINT (529377.7846075087 183993.2942823111)
2997,113205,113205,6.57089E+11,E09000007,Camden,Inner London - West,3251736000,1 LAWFORDS WHARF,LYME STREET,LONDON,...,D,NaN,E05000130,NaN,2012-03-19,0,CO,203.0,NaN,POINT (529185.7852216109 184064.2957215998)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1490,85831,85831,5.35006E+11,E09000007,Camden,Inner London - West,99271063,GND F 57,CAMDEN ROAD,LONDON,...,D,NaN,E05000131,NaN,2012-03-19,3,CS2,24.0,NaN,POINT (529196.7824470752 184232.3021180349)
3250,113502,113502,6.61009E+11,E09000007,Camden,Inner London - West,110608063,ARCHES 90 & 91,BAYNES STREET,LONDON,...,D,NaN,E05000131,NaN,2012-03-19,0,CG1,289.0,NaN,POINT (529354.7875236059 184154.2926299943)
874,82707,82707,5.12019E+11,E09000007,Camden,Inner London - West,3386993000,2ND F 187-191,ROYAL COLLEGE STREET,LONDON,...,C,NaN,E05000130,NaN,NaN,0,CO,203.0,NaN,POINT (529258.7831391912 184058.2975567151)
3426,123839,123839,0074200631007B,E09000007,Camden,Inner London - West,95657063,UNIT 5 AT 63,PRATT STREET,LONDON,...,D,NaN,E05000143,NaN,2013-10-22,0,IF3,96.0,NaN,POINT (529337.7843255643 183958.291385453)


In [1779]:
#deleted those cols columns from the dataset as they only contain nulls

Kentish = Kentish.drop(cols, axis=1)

In [1780]:
Kentish['new_date'] = pd.to_datetime(Kentish['status_date'], errors='coerce')

In [1781]:
Kentish = Kentish.sort_values(by= ['new_date'])

In [1782]:
Kentish['new_date'].loc[(Kentish['new_date'] < '2010-04-01')] = '2010-04-01'

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [1783]:
Kentish = Kentish.sort_values(by= ['location_code', 'period'])

In [1784]:
Kentish['locaton_code'] = Kentish['location_code'].astype("category")

In [1785]:
locations = Kentish['location_code'].unique()

In [1786]:
Kentish.loc[Kentish['name'].isnull(), 'name'] = Kentish['location_code']

In [1787]:
Kentish['status_date'] = pd.to_datetime(Kentish['status_date'], errors='coerce')

In [1788]:
Kentish['status_date'].iloc[Kentish['status_date'] < '2010-04-01'] = '04/01/2010'

In [1789]:
Kentish['status_date'] = pd.to_datetime(Kentish['status_date'], errors='coerce')

In [1790]:
Kentish['period2'] = pd.to_datetime(Kentish['period'])

In [1791]:
Kentish['period2'] = Kentish['period2'] + pd.DateOffset(months=1)

In [1792]:
Kentish['duration'] = Kentish['period2'] - Kentish['status_date']

In [1793]:
Kentish['duration'] = Kentish['duration'].dt.days

In [1794]:
Kentish['study-start'] = '04/01/2010'

In [1795]:
Kentish['study-start'] = pd.to_datetime(Kentish['study-start'], errors='coerce')

In [1796]:
Kentish['time-of-entry'] = Kentish['status_date'] -  Kentish['study-start']

In [1797]:
Kentish['time-at-open'] = 0

In [1798]:
Kentish['vacant'] = 1

In [1799]:
Kentish = Kentish.drop(Kentish[(Kentish['status'] == True)].index)

In [1800]:
Kentish['location_code'] = Kentish['location_code'].astype("category")

In [1801]:
locations = list(Kentish['location_code'].unique())

In [1802]:
Kentish = Kentish.set_index('location_code')

In [1803]:
Kentish['survival'] = 0

In [1804]:
Kentish['survival'] = Kentish['duration']

In [1805]:
Kentish['entry'] = Kentish['time-of-entry'].dt.days

In [1806]:
Kentish['time-at-open'] = Kentish['survival'] + Kentish['entry']

In [1807]:
Kentish['stat'] = Kentish['status'].astype(int)

In [1808]:
Kentish["StatusChange"] =  Kentish['stat'].shift() != Kentish['stat']

In [1809]:
Kentish["StatusChange"].iloc[0] = False

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [1810]:
Kentish["TenantChange"] =  Kentish['name'].shift(-1) != Kentish['name'] 

In [1811]:
Kentish["TenantChange"].iloc[0] = False

In [1812]:
counter = 0 
stoppingCondition = len(locations)
Kentish2 = pd.DataFrame(data=None, columns=Kentish.columns, index=None)

while counter < stoppingCondition:
        i = locations[counter]
        m = Kentish.loc[i, :].iloc[[ Kentish.loc[i, :]['status_duration'].argmax()]]
        Kentish2 = Kentish2.append(m, ignore_index=False)
        counter += 1

In [1813]:
Kentish3 = Kentish2.drop_duplicates()

In [1814]:
Kentish3['highstreet'] = 'Kentish'

In [1815]:
Kentish3.loc[Kentish3['time-at-open'] == 4018, ['vacant']] = 0

In [1816]:
Kentish3.to_csv('Kentish-vacant-survival.csv')

#### 3. Fortress 

In [1817]:
Fortress = Fortress.sort_values(by= ['status_date'])

In [1818]:
cols = ['V3', 'V4', 'V5', 'V13', 'V14', 'V15', 'V16', 'V17', 'V19', 'V20', 'V21', 'V26', 'V28', 'V30', 'V36', 'V38', 'V39', 'V48', 'V51', 'V54', 'V55', 'V56', 'V57', 
        'V58', 'V59', 'V60', 'V64', 'V70', 'V72', 'V77']

In [1819]:
#deleted those cols columns from the dataset as they only contain nulls

Fortress = Fortress.drop(cols, axis=1)

In [1820]:
Fortress['new_date'] = pd.to_datetime(Fortress['status_date'], errors='coerce')

In [1821]:
Fortress = Fortress.sort_values(by= ['new_date'])

In [1822]:
Fortress['new_date'].loc[(Fortress['new_date'] < '2010-04-01')] = '2010-04-01'

In [1823]:
Fortress = Fortress.sort_values(by= ['location_code', 'period'])

In [1824]:
Fortress['locaton_code'] = Fortress['location_code'].astype("category")

In [1825]:
locations = Fortress['location_code'].unique()

In [1826]:
Fortress.loc[Fortress['name'].isnull(), 'name'] = Fortress['location_code']

In [1827]:
Fortress['status_date'] = pd.to_datetime(Fortress['status_date'], errors='coerce')

In [1828]:
Fortress['status_date'].iloc[Fortress['status_date'] < '2010-04-01'] = '04/01/2010'

In [1829]:
Fortress['status_date'] = pd.to_datetime(Fortress['status_date'], errors='coerce')

In [1830]:
Fortress['period2'] = pd.to_datetime(Fortress['period'])

In [1831]:
Fortress['period2'] = Fortress['period2'] + pd.DateOffset(months=1)

In [1832]:
Fortress['duration'] = Fortress['period2'] - Fortress['status_date']

In [1833]:
Fortress['duration'] = Fortress['duration'].dt.days

In [1834]:
Fortress['study-start'] = '04/01/2010'

In [1835]:
Fortress['study-start'] = pd.to_datetime(Fortress['study-start'], errors='coerce')

In [1836]:
Fortress['time-of-entry'] = Fortress['status_date'] -  Fortress['study-start']

In [1837]:
Fortress['time-at-open'] = 0

In [1838]:
Fortress['vacant'] = 1

In [1839]:
Fortress = Fortress.drop(Fortress[(Fortress['status'] == True)].index)

In [1840]:
Fortress['location_code'] = Fortress['location_code'].astype("category")

In [1841]:
locations = list(Fortress['location_code'].unique())

In [1842]:
Fortress = Fortress.set_index('location_code')

In [1843]:
Fortress['survival'] = 0

In [1844]:
Fortress['survival'] = Fortress['duration']

In [1845]:
Fortress['entry'] = Fortress['time-of-entry'].dt.days

In [1846]:
Fortress['time-at-open'] = Fortress['survival'] + Fortress['entry']

In [1847]:
Fortress['stat'] = Fortress['status'].astype(int)

In [1848]:
Fortress["StatusChange"] =  Fortress['stat'].shift() != Fortress['stat']

In [1849]:
Fortress["StatusChange"].iloc[0] = False

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [1850]:
Fortress["TenantChange"] =  Fortress['name'].shift(-1) != Fortress['name'] 

In [1851]:
Fortress["TenantChange"].iloc[0] = False

In [1852]:
counter = 0 
stoppingCondition = len(locations)
Fortress2 = pd.DataFrame(data=None, columns=Fortress.columns, index=None)

while counter < stoppingCondition:
        i = locations[counter]
        m = Fortress.loc[i, :].iloc[[ Fortress.loc[i, :]['status_duration'].argmax()]]
        Fortress2 = Fortress2.append(m, ignore_index=False)
        counter += 1

In [1853]:
Fortress3 = Fortress2.drop_duplicates()

In [1854]:
Fortress3.loc[Fortress3['time-at-open'] == 4018, ['vacant']] = 0

In [1855]:
Fortress3['highstreet'] = 'Fortress'

In [1856]:
Fortress3.to_csv('Fortress-vacant-survival.csv')

#### 4. Camden Town

In [1857]:
CamdenT = CamdenT.sort_values(by= ['status_date'])

In [1858]:
cols = ['V3', 'V4', 'V5', 'V13', 'V14', 'V15', 'V16', 'V17', 'V19', 'V20', 'V21', 'V26', 'V28', 'V30', 'V36', 'V38', 'V39', 'V48', 'V51', 'V54', 'V55', 'V56', 'V57', 
        'V58', 'V59', 'V60', 'V64', 'V70', 'V72', 'V77']

In [1859]:
#deleted those cols columns from the dataset as they only contain nulls

CamdenT = CamdenT.drop(cols, axis=1)

In [1860]:
CamdenT['new_date'] = pd.to_datetime(CamdenT['status_date'], errors='coerce')

In [1861]:
CamdenT = CamdenT.sort_values(by= ['new_date'])

In [1862]:
CamdenT['new_date'].loc[(CamdenT['new_date'] < '2010-04-01')] = '2010-04-01'

In [1863]:
CamdenT = CamdenT.sort_values(by= ['location_code', 'period'])

In [1864]:
CamdenT['locaton_code'] = CamdenT['location_code'].astype("category")

In [1865]:
locations = CamdenT['location_code'].unique()

In [1866]:
CamdenT.loc[CamdenT['name'].isnull(), 'name'] = CamdenT['location_code']

In [1867]:
CamdenT['status_date'] = pd.to_datetime(CamdenT['status_date'], errors='coerce')

In [1868]:
CamdenT['status_date'].iloc[CamdenT['status_date'] < '2010-04-01'] = '04/01/2010'

In [1869]:
CamdenT['status_date'] = pd.to_datetime(CamdenT['status_date'], errors='coerce')

In [1870]:
CamdenT['period2'] = pd.to_datetime(CamdenT['period'])

In [1871]:
CamdenT['period2'] = CamdenT['period2'] + pd.DateOffset(months=1)

In [1872]:
CamdenT['duration'] = CamdenT['period2'] - CamdenT['status_date']

In [1873]:
CamdenT['duration'] = CamdenT['duration'].dt.days

In [1874]:
CamdenT['study-start'] = '04/01/2010'

In [1875]:
CamdenT['study-start'] = pd.to_datetime(CamdenT['study-start'], errors='coerce')

In [1876]:
CamdenT['time-of-entry'] = CamdenT['status_date'] -  CamdenT['study-start']

In [1877]:
CamdenT['time-at-open'] = 0

In [1878]:
CamdenT['vacant'] = 1

In [1879]:
CamdenT = CamdenT.drop(CamdenT[(CamdenT['status'] == True)].index)

In [1880]:
CamdenT['location_code'] = CamdenT['location_code'].astype("category")

In [1881]:
locations = list(CamdenT['location_code'].unique())

In [1882]:
CamdenT = CamdenT.set_index('location_code')

In [1883]:
CamdenT['survival'] = 0

In [1884]:
CamdenT['survival'] = CamdenT['duration']

In [1885]:
CamdenT['entry'] = CamdenT['time-of-entry'].dt.days

In [1886]:
CamdenT['time-at-open'] = CamdenT['survival'] + CamdenT['entry']

In [1887]:
CamdenT['stat'] = CamdenT['status'].astype(int)

In [1888]:
CamdenT["StatusChange"] =  CamdenT['stat'].shift() != CamdenT['stat']

In [1889]:
CamdenT["StatusChange"].iloc[0] = False

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [1890]:
CamdenT["TenantChange"] =  CamdenT['name'].shift(-1) != CamdenT['name'] 

In [1891]:
CamdenT["TenantChange"].iloc[0] = False

In [1892]:
counter = 0 
stoppingCondition = len(locations)
CamdenT2 = pd.DataFrame(data=None, columns=CamdenT.columns, index=None)

while counter < stoppingCondition:
        i = locations[counter]
        m = CamdenT.loc[i, :].iloc[[ CamdenT.loc[i, :]['status_duration'].argmax()]]
        CamdenT2 = CamdenT2.append(m, ignore_index=False)
        counter += 1

In [1893]:
CamdenT3 = CamdenT2.drop_duplicates()

In [1894]:
CamdenT3.loc[CamdenT3['time-at-open'] == 4018, ['vacant']] = 0

In [1895]:
CamdenT3['highstreet'] = 'CamdenT'

In [1896]:
CamdenT3.to_csv('CamdenT-vacant-survival.csv')

#### 5. Belsize

In [1897]:
Belsize = Belsize.sort_values(by= ['status_date'])

In [1898]:
cols = ['V3', 'V4', 'V5', 'V13', 'V14', 'V15', 'V16', 'V17', 'V19', 'V20', 'V21', 'V26', 'V28', 'V30', 'V36', 'V38', 'V39', 'V48', 'V51', 'V54', 'V55', 'V56', 'V57', 
        'V58', 'V59', 'V60', 'V64', 'V70', 'V72', 'V77']

In [1899]:
#deleted those cols columns from the dataset as they only contain nulls

Belsize = Belsize.drop(cols, axis=1)

In [1900]:
Belsize['new_date'] = pd.to_datetime(Belsize['status_date'], errors='coerce')

In [1901]:
Belsize = Belsize.sort_values(by= ['new_date'])

In [1902]:
Belsize['new_date'].loc[(Belsize['new_date'] < '2010-04-01')] = '2010-04-01'

In [1903]:
Belsize = Belsize.sort_values(by= ['location_code', 'period'])

In [1904]:
Belsize['locaton_code'] = Belsize['location_code'].astype("category")

In [1905]:
locations = Belsize['location_code'].unique()

In [1906]:
Belsize.loc[Belsize['name'].isnull(), 'name'] = Belsize['location_code']

In [1907]:
Belsize['status_date'] = pd.to_datetime(Belsize['status_date'], errors='coerce')

In [1908]:
Belsize['status_date'].iloc[Belsize['status_date'] < '2010-04-01'] = '04/01/2010'

In [1909]:
Belsize['status_date'] = pd.to_datetime(Belsize['status_date'], errors='coerce')

In [1910]:
Belsize['period2'] = pd.to_datetime(Belsize['period'])

In [1911]:
Belsize['period2'] = Belsize['period2'] + pd.DateOffset(months=1)

In [1912]:
Belsize['duration'] = Belsize['period2'] - Belsize['status_date']

In [1913]:
Belsize['duration'] = Belsize['duration'].dt.days

In [1914]:
Belsize['study-start'] = '04/01/2010'

In [1915]:
Belsize['study-start'] = pd.to_datetime(Belsize['study-start'], errors='coerce')

In [1916]:
Belsize['time-of-entry'] = Belsize['status_date'] -  Belsize['study-start']

In [1917]:
Belsize['time-at-open'] = 0

In [1918]:
Belsize['vacant'] = 1

In [1919]:
Belsize = Belsize.drop(Belsize[(Belsize['status'] == True)].index)

In [1920]:
Belsize['location_code'] = Belsize['location_code'].astype("category")

In [1921]:
locations = list(Belsize['location_code'].unique())

In [1922]:
Belsize = Belsize.set_index('location_code')

In [1923]:
Belsize['survival'] = 0

In [1924]:
Belsize['survival'] = Belsize['duration']

In [1925]:
Belsize['entry'] = Belsize['time-of-entry'].dt.days

In [1926]:
Belsize['time-at-open'] = Belsize['survival'] + Belsize['entry']

In [1927]:
Belsize['stat'] = Belsize['status'].astype(int)

In [1928]:
Belsize["StatusChange"] =  Belsize['stat'].shift() != Belsize['stat']

In [1929]:
Belsize["StatusChange"].iloc[0] = False

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [1930]:
Belsize["TenantChange"] =  Belsize['name'].shift(-1) != Belsize['name'] 

In [1931]:
Belsize["TenantChange"].iloc[0] = False

In [1932]:
counter = 0 
stoppingCondition = len(locations)
Belsize2 = pd.DataFrame(data=None, columns=Belsize.columns, index=None)

while counter < stoppingCondition:
        i = locations[counter]
        m = Belsize.loc[i, :].iloc[[ Belsize.loc[i, :]['status_duration'].argmax()]]
        Belsize2 = Belsize2.append(m, ignore_index=False)
        counter += 1

In [1933]:
Belsize3 = Belsize2.drop_duplicates()

In [1934]:
Belsize3.loc[Belsize3['time-at-open'] == 4018, ['vacant']] = 0

In [1935]:
Belsize3['highstreet'] = 'Belsize'

In [1936]:
Belsize3.to_csv('Belsize-vacant-survival.csv')

#### 6. Loudoun

In [1937]:
Loudoun = Loudoun.sort_values(by= ['status_date'])

In [1938]:
cols = ['V3', 'V4', 'V5', 'V13', 'V14', 'V15', 'V16', 'V17', 'V19', 'V20', 'V21', 'V26', 'V28', 'V30', 'V36', 'V38', 'V39', 'V48', 'V51', 'V54', 'V55', 'V56', 'V57', 
        'V58', 'V59', 'V60', 'V64', 'V70', 'V72', 'V77']

In [1939]:
#deleted those cols columns from the dataset as they only contain nulls

Loudoun = Loudoun.drop(cols, axis=1)

In [1940]:
Loudoun['new_date'] = pd.to_datetime(Loudoun['status_date'], errors='coerce')

In [1941]:
Loudoun = Loudoun.sort_values(by= ['new_date'])

In [1942]:
Loudoun['new_date'].loc[(Loudoun['new_date'] < '2010-04-01')] = '2010-04-01'

In [1943]:
Loudoun = Loudoun.sort_values(by= ['location_code', 'period'])

In [1944]:
Loudoun['locaton_code'] = Loudoun['location_code'].astype("category")

In [1945]:
locations = Loudoun['location_code'].unique()

In [1946]:
Loudoun.loc[Loudoun['name'].isnull(), 'name'] = Loudoun['location_code']

In [1947]:
Loudoun['status_date'] = pd.to_datetime(Loudoun['status_date'], errors='coerce')

In [1948]:
Loudoun['status_date'].iloc[Loudoun['status_date'] < '2010-04-01'] = '04/01/2010'

In [1949]:
Loudoun['status_date'] = pd.to_datetime(Loudoun['status_date'], errors='coerce')

In [1950]:
Loudoun['period2'] = pd.to_datetime(Loudoun['period'])

In [1951]:
Loudoun['period2'] = Loudoun['period2'] + pd.DateOffset(months=1)

In [1952]:
Loudoun['duration'] = Loudoun['period2'] - Loudoun['status_date']

In [1953]:
Loudoun['duration'] = Loudoun['duration'].dt.days

In [1954]:
Loudoun['study-start'] = '04/01/2010'

In [1955]:
Loudoun['study-start'] = pd.to_datetime(Loudoun['study-start'], errors='coerce')

In [1956]:
Loudoun['time-of-entry'] = Loudoun['status_date'] -  Loudoun['study-start']

In [1957]:
Loudoun['time-at-open'] = 0

In [1958]:
Loudoun['vacant'] = 1

In [1959]:
Loudoun = Loudoun.drop(Loudoun[(Loudoun['status'] == True)].index)

In [1960]:
Loudoun['location_code'] = Loudoun['location_code'].astype("category")

In [1961]:
locations = list(Loudoun['location_code'].unique())

In [1962]:
Loudoun = Loudoun.set_index('location_code')

In [1963]:
Loudoun['survival'] = 0

In [1964]:
Loudoun['survival'] = Loudoun['duration']

In [1965]:
Loudoun['entry'] = Loudoun['time-of-entry'].dt.days

In [1966]:
Loudoun['time-at-open'] = Loudoun['survival'] + Loudoun['entry']

In [1967]:
Loudoun['stat'] = Loudoun['status'].astype(int)

In [1968]:
Loudoun["StatusChange"] =  Loudoun['stat'].shift() != Loudoun['stat']

In [1969]:
Loudoun["StatusChange"].iloc[0] = False

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [1970]:
Loudoun["TenantChange"] =  Loudoun['name'].shift(-1) != Loudoun['name'] 

In [1971]:
Loudoun["TenantChange"].iloc[0] = False

In [1972]:
counter = 0 
stoppingCondition = len(locations)
Loudoun2 = pd.DataFrame(data=None, columns=Loudoun.columns, index=None)

while counter < stoppingCondition:
        i = locations[counter]
        m = Loudoun.loc[i, :].iloc[[ Loudoun.loc[i, :]['status_duration'].argmax()]]
        Loudoun2 = Loudoun2.append(m, ignore_index=False)
        counter += 1

In [1973]:
Loudoun3 = Loudoun2.drop_duplicates()

In [1974]:
Loudoun3.loc[Loudoun3['time-at-open'] == 4018, ['vacant']] = 0

In [1975]:
Loudoun3['highstreet'] = 'Loudoun'

In [1976]:
Loudoun3.to_csv('Loudoun-vacant-survival.csv')

#### 7. Haverstock

In [1977]:
Haverstock = Haverstock.sort_values(by= ['status_date'])

In [1978]:
cols = ['V3', 'V4', 'V5', 'V13', 'V14', 'V15', 'V16', 'V17', 'V19', 'V20', 'V21', 'V26', 'V28', 'V30', 'V36', 'V38', 'V39', 'V48', 'V51', 'V54', 'V55', 'V56', 'V57', 
        'V58', 'V59', 'V60', 'V64', 'V70', 'V72', 'V77']

In [1979]:
#deleted those cols columns from the dataset as they only contain nulls

Haverstock = Haverstock.drop(cols, axis=1)

In [1980]:
Haverstock['new_date'] = pd.to_datetime(Haverstock['status_date'], errors='coerce')

In [1981]:
Haverstock = Haverstock.sort_values(by= ['new_date'])

In [1982]:
Haverstock['new_date'].loc[(Haverstock['new_date'] < '2010-04-01')] = '2010-04-01'

In [1983]:
Haverstock = Haverstock.sort_values(by= ['location_code', 'period'])

In [1984]:
Haverstock['locaton_code'] = Haverstock['location_code'].astype("category")

In [1985]:
locations = Haverstock['location_code'].unique()

In [1986]:
Haverstock.loc[Haverstock['name'].isnull(), 'name'] = Haverstock['location_code']

In [1987]:
Haverstock['status_date'] = pd.to_datetime(Haverstock['status_date'], errors='coerce')

In [1988]:
Haverstock['status_date'].iloc[Haverstock['status_date'] < '2010-04-01'] = '04/01/2010'

In [1989]:
Haverstock['status_date'] = pd.to_datetime(Haverstock['status_date'], errors='coerce')

In [1990]:
Haverstock['period2'] = pd.to_datetime(Haverstock['period'])

In [1991]:
Haverstock['period2'] = Haverstock['period2'] + pd.DateOffset(months=1)

In [1992]:
Haverstock['duration'] = Haverstock['period2'] - Haverstock['status_date']

In [1993]:
Haverstock['duration'] = Haverstock['duration'].dt.days

In [1994]:
Haverstock['study-start'] = '04/01/2010'

In [1995]:
Haverstock['study-start'] = pd.to_datetime(Haverstock['study-start'], errors='coerce')

In [1996]:
Haverstock['time-of-entry'] = Haverstock['status_date'] -  Haverstock['study-start']

In [1997]:
Haverstock['time-at-open'] = 0

In [1998]:
Haverstock['vacant'] = 1

In [1999]:
Haverstock = Haverstock.drop(Haverstock[(Haverstock['status'] == True)].index)

In [2000]:
Haverstock['location_code'] = Haverstock['location_code'].astype("category")

In [2001]:
locations = list(Haverstock['location_code'].unique())

In [2002]:
Haverstock = Haverstock.set_index('location_code')

In [2003]:
Haverstock['survival'] = 0

In [2004]:
Haverstock['survival'] = Haverstock['duration']

In [2005]:
Haverstock['entry'] = Haverstock['time-of-entry'].dt.days

In [2006]:
Haverstock['time-at-open'] = Haverstock['survival'] + Haverstock['entry']

In [2007]:
Haverstock['stat'] = Haverstock['status'].astype(int)

In [2008]:
Haverstock["StatusChange"] =  Haverstock['stat'].shift() != Haverstock['stat']

In [2009]:
Haverstock["StatusChange"].iloc[0] = False

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [2010]:
Haverstock["TenantChange"] =  Haverstock['name'].shift(-1) != Haverstock['name'] 

In [2011]:
Haverstock["TenantChange"].iloc[0] = False

In [2012]:
counter = 0 
stoppingCondition = len(locations)
Haverstock2 = pd.DataFrame(data=None, columns=Haverstock.columns, index=None)

while counter < stoppingCondition:
        i = locations[counter]
        m = Haverstock.loc[i, :].iloc[[ Haverstock.loc[i, :]['status_duration'].argmax()]]
        Haverstock2 = Haverstock2.append(m, ignore_index=False)
        counter += 1

In [2013]:
Haverstock3 = Haverstock2.drop_duplicates()

In [2014]:
Haverstock3.loc[Haverstock3['time-at-open'] == 4018, ['vacant']] = 0

In [2015]:
Haverstock3['highstreet'] = 'Haverstock'

In [2016]:
Haverstock3.to_csv('Haverstock-vacant-survival.csv')

#### 8. SouthEnd

In [2017]:
SouthEnd = SouthEnd.sort_values(by= ['status_date'])

In [2018]:
cols = ['V3', 'V4', 'V5', 'V13', 'V14', 'V15', 'V16', 'V17', 'V19', 'V20', 'V21', 'V26', 'V28', 'V30', 'V36', 'V38', 'V39', 'V48', 'V51', 'V54', 'V55', 'V56', 'V57', 
        'V58', 'V59', 'V60', 'V64', 'V70', 'V72', 'V77']

In [2019]:
#deleted those cols columns from the dataset as they only contain nulls

SouthEnd = SouthEnd.drop(cols, axis=1)

In [2020]:
SouthEnd['new_date'] = pd.to_datetime(SouthEnd['status_date'], errors='coerce')

In [2021]:
SouthEnd = SouthEnd.sort_values(by= ['new_date'])

In [2022]:
SouthEnd['new_date'].loc[(SouthEnd['new_date'] < '2010-04-01')] = '2010-04-01'

In [2023]:
SouthEnd = SouthEnd.sort_values(by= ['location_code', 'period'])

In [2024]:
SouthEnd['locaton_code'] = SouthEnd['location_code'].astype("category")

In [2025]:
locations = SouthEnd['location_code'].unique()

In [2026]:
SouthEnd.loc[SouthEnd['name'].isnull(), 'name'] = SouthEnd['location_code']

In [2027]:
SouthEnd['status_date'] = pd.to_datetime(SouthEnd['status_date'], errors='coerce')

In [2028]:
SouthEnd['status_date'].iloc[SouthEnd['status_date'] < '2010-04-01'] = '04/01/2010'

In [2029]:
SouthEnd['status_date'] = pd.to_datetime(SouthEnd['status_date'], errors='coerce')

In [2030]:
SouthEnd['period2'] = pd.to_datetime(SouthEnd['period'])

In [2031]:
SouthEnd['period2'] = SouthEnd['period2'] + pd.DateOffset(months=1)

In [2032]:
SouthEnd['duration'] = SouthEnd['period2'] - SouthEnd['status_date']

In [2033]:
SouthEnd['duration'] = SouthEnd['duration'].dt.days

In [2034]:
SouthEnd['study-start'] = '04/01/2010'

In [2035]:
SouthEnd['study-start'] = pd.to_datetime(SouthEnd['study-start'], errors='coerce')

In [2036]:
SouthEnd['time-of-entry'] = SouthEnd['status_date'] -  SouthEnd['study-start']

In [2037]:
SouthEnd['time-at-open'] = 0

In [2038]:
SouthEnd['vacant'] = 1

In [2039]:
SouthEnd = SouthEnd.drop(SouthEnd[(SouthEnd['status'] == True)].index)

In [2040]:
SouthEnd['location_code'] = SouthEnd['location_code'].astype("category")

In [2041]:
locations = list(SouthEnd['location_code'].unique())

In [2042]:
SouthEnd = SouthEnd.set_index('location_code')

In [2043]:
SouthEnd['survival'] = 0

In [2044]:
SouthEnd['survival'] = SouthEnd['duration']

In [2045]:
SouthEnd['entry'] = SouthEnd['time-of-entry'].dt.days

In [2046]:
SouthEnd['time-at-open'] = SouthEnd['survival'] + SouthEnd['entry']

In [2047]:
SouthEnd['stat'] = SouthEnd['status'].astype(int)

In [2048]:
SouthEnd["StatusChange"] =  SouthEnd['stat'].shift() != SouthEnd['stat']

In [2049]:
SouthEnd["StatusChange"].iloc[0] = False

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [2050]:
SouthEnd["TenantChange"] =  SouthEnd['name'].shift(-1) != SouthEnd['name'] 

In [2051]:
SouthEnd["TenantChange"].iloc[0] = False

In [2052]:
counter = 0 
stoppingCondition = len(locations)
SouthEnd2 = pd.DataFrame(data=None, columns=SouthEnd.columns, index=None)

while counter < stoppingCondition:
        i = locations[counter]
        m = SouthEnd.loc[i, :].iloc[[ SouthEnd.loc[i, :]['status_duration'].argmax()]]
        SouthEnd2 = SouthEnd2.append(m, ignore_index=False)
        counter += 1

In [2053]:
SouthEnd3 = SouthEnd2.drop_duplicates()

In [2054]:
SouthEnd3.loc[SouthEnd3['time-at-open'] == 4018, ['vacant']] = 0

In [2055]:
SouthEnd3['highstreet'] = 'SouthEnd'

In [2056]:
SouthEnd3.to_csv('SouthEnd-vacant-survival.csv')

#### 9. Cricklewood

In [2057]:
Cricklewood = Cricklewood.sort_values(by= ['status_date'])

In [2058]:
cols = ['V3', 'V4', 'V5', 'V13', 'V14', 'V15', 'V16', 'V17', 'V19', 'V20', 'V21', 'V26', 'V28', 'V30', 'V36', 'V38', 'V39', 'V48', 'V51', 'V54', 'V55', 'V56', 'V57', 
        'V58', 'V59', 'V60', 'V64', 'V70', 'V72', 'V77']

In [2059]:
#deleted those cols columns from the dataset as they only contain nulls

Cricklewood = Cricklewood.drop(cols, axis=1)

In [2060]:
Cricklewood['new_date'] = pd.to_datetime(Cricklewood['status_date'], errors='coerce')

In [2061]:
Cricklewood = Cricklewood.sort_values(by= ['new_date'])

In [2062]:
Cricklewood['new_date'].loc[(Cricklewood['new_date'] < '2010-04-01')] = '2010-04-01'

In [2063]:
Cricklewood = Cricklewood.sort_values(by= ['location_code', 'period'])

In [2064]:
Cricklewood['locaton_code'] = Cricklewood['location_code'].astype("category")

In [2065]:
locations = Cricklewood['location_code'].unique()

In [2066]:
Cricklewood.loc[Cricklewood['name'].isnull(), 'name'] = Cricklewood['location_code']

In [2067]:
Cricklewood['status_date'] = pd.to_datetime(Cricklewood['status_date'], errors='coerce')

In [2068]:
Cricklewood['status_date'].iloc[Cricklewood['status_date'] < '2010-04-01'] = '04/01/2010'

In [2069]:
Cricklewood['status_date'] = pd.to_datetime(Cricklewood['status_date'], errors='coerce')

In [2070]:
Cricklewood['period2'] = pd.to_datetime(Cricklewood['period'])

In [2071]:
Cricklewood['period2'] = Cricklewood['period2'] + pd.DateOffset(months=1)

In [2072]:
Cricklewood['duration'] = Cricklewood['period2'] - Cricklewood['status_date']

In [2073]:
Cricklewood['duration'] = Cricklewood['duration'].dt.days

In [2074]:
Cricklewood['study-start'] = '04/01/2010'

In [2075]:
Cricklewood['study-start'] = pd.to_datetime(Cricklewood['study-start'], errors='coerce')

In [2076]:
Cricklewood['time-of-entry'] = Cricklewood['status_date'] -  Cricklewood['study-start']

In [2077]:
Cricklewood['time-at-open'] = 0

In [2078]:
Cricklewood['vacant'] = 1

In [2079]:
Cricklewood = Cricklewood.drop(Cricklewood[(Cricklewood['status'] == True)].index)

In [2080]:
Cricklewood['location_code'] = Cricklewood['location_code'].astype("category")

In [2081]:
locations = list(Cricklewood['location_code'].unique())

In [2082]:
Cricklewood = Cricklewood.set_index('location_code')

In [2083]:
Cricklewood['survival'] = 0

In [2084]:
Cricklewood['survival'] = Cricklewood['duration']

In [2085]:
Cricklewood['entry'] = Cricklewood['time-of-entry'].dt.days

In [2086]:
Cricklewood['time-at-open'] = Cricklewood['survival'] + Cricklewood['entry']

In [2087]:
Cricklewood['stat'] = Cricklewood['status'].astype(int)

In [2088]:
Cricklewood["StatusChange"] =  Cricklewood['stat'].shift() != Cricklewood['stat']

In [2089]:
Cricklewood["StatusChange"].iloc[0] = False

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [2090]:
Cricklewood["TenantChange"] =  Cricklewood['name'].shift(-1) != Cricklewood['name'] 

In [2091]:
Cricklewood["TenantChange"].iloc[0] = False

In [2092]:
counter = 0 
stoppingCondition = len(locations)
Cricklewood2 = pd.DataFrame(data=None, columns=Cricklewood.columns, index=None)

while counter < stoppingCondition:
        i = locations[counter]
        m = Cricklewood.loc[i, :].iloc[[ Cricklewood.loc[i, :]['status_duration'].argmax()]]
        Cricklewood2 = Cricklewood2.append(m, ignore_index=False)
        counter += 1

In [2093]:
Cricklewood3 = Cricklewood2.drop_duplicates()

In [2094]:
Cricklewood3.loc[Cricklewood3['time-at-open'] == 4018, ['vacant']] = 0

In [2095]:
Cricklewood3['highstreet'] = 'Cricklewood'

In [2096]:
Cricklewood3.to_csv('Cricklewood-vacant-survival.csv')

#### 10. Regents

In [2097]:
Regents = Regents.sort_values(by= ['status_date'])

In [2098]:
cols = ['V3', 'V4', 'V5', 'V13', 'V14', 'V15', 'V16', 'V17', 'V19', 'V20', 'V21', 'V26', 'V28', 'V30', 'V36', 'V38', 'V39', 'V48', 'V51', 'V54', 'V55', 'V56', 'V57', 
        'V58', 'V59', 'V60', 'V64', 'V70', 'V72', 'V77']

In [2099]:
#deleted those cols columns from the dataset as they only contain nulls

Regents = Regents.drop(cols, axis=1)

In [2100]:
Regents['new_date'] = pd.to_datetime(Regents['status_date'], errors='coerce')

In [2101]:
Regents = Regents.sort_values(by= ['new_date'])

In [2102]:
Regents['new_date'].loc[(Regents['new_date'] < '2010-04-01')] = '2010-04-01'

In [2103]:
Regents = Regents.sort_values(by= ['location_code', 'period'])

In [2104]:
Regents['locaton_code'] = Regents['location_code'].astype("category")

In [2105]:
locations = Regents['location_code'].unique()

In [2106]:
Regents.loc[Regents['name'].isnull(), 'name'] = Regents['location_code']

In [2107]:
Regents['status_date'] = pd.to_datetime(Regents['status_date'], errors='coerce')

In [2108]:
Regents['status_date'].iloc[Regents['status_date'] < '2010-04-01'] = '04/01/2010'

In [2109]:
Regents['status_date'] = pd.to_datetime(Regents['status_date'], errors='coerce')

In [2110]:
Regents['period2'] = pd.to_datetime(Regents['period'])

In [2111]:
Regents['period2'] = Regents['period2'] + pd.DateOffset(months=1)

In [2112]:
Regents['duration'] = Regents['period2'] - Regents['status_date']

In [2113]:
Regents['duration'] = Regents['duration'].dt.days

In [2114]:
Regents['study-start'] = '04/01/2010'

In [2115]:
Regents['study-start'] = pd.to_datetime(Regents['study-start'], errors='coerce')

In [2116]:
Regents['time-of-entry'] = Regents['status_date'] -  Regents['study-start']

In [2117]:
Regents['time-at-open'] = 0

In [2118]:
Regents['vacant'] = 1

In [2119]:
Regents = Regents.drop(Regents[(Regents['status'] == True)].index)

In [2120]:
Regents['location_code'] = Regents['location_code'].astype("category")

In [2121]:
locations = list(Regents['location_code'].unique())

In [2122]:
Regents = Regents.set_index('location_code')

In [2123]:
Regents['survival'] = 0

In [2124]:
Regents['survival'] = Regents['duration']

In [2125]:
Regents['entry'] = Regents['time-of-entry'].dt.days

In [2126]:
Regents['time-at-open'] = Regents['survival'] + Regents['entry']

In [2127]:
Regents['stat'] = Regents['status'].astype(int)

In [2128]:
Regents["StatusChange"] =  Regents['stat'].shift() != Regents['stat']

In [2129]:
Regents["StatusChange"].iloc[0] = False

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [2130]:
Regents["TenantChange"] =  Regents['name'].shift(-1) != Regents['name'] 

In [2131]:
Regents["TenantChange"].iloc[0] = False

In [2132]:
counter = 0 
stoppingCondition = len(locations)
Regents2 = pd.DataFrame(data=None, columns=Regents.columns, index=None)

while counter < stoppingCondition:
        i = locations[counter]
        m = Regents.loc[i, :].iloc[[ Regents.loc[i, :]['status_duration'].argmax()]]
        Regents2 = Regents2.append(m, ignore_index=False)
        counter += 1

In [2133]:
Regents3 = Regents2.drop_duplicates()

In [2134]:
Regents3.loc[Regents3['time-at-open'] == 4018, ['vacant']] = 0

In [2135]:
Regents3['highstreet'] = 'Regents'

In [2136]:
Regents3.to_csv('Regents-vacant-survival.csv')

#### 11. Queens

In [2137]:
Queens = Queens.sort_values(by= ['status_date'])

In [2138]:
cols = ['V3', 'V4', 'V5', 'V13', 'V14', 'V15', 'V16', 'V17', 'V19', 'V20', 'V21', 'V26', 'V28', 'V30', 'V36', 'V38', 'V39', 'V48', 'V51', 'V54', 'V55', 'V56', 'V57', 
        'V58', 'V59', 'V60', 'V64', 'V70', 'V72', 'V77']

In [2139]:
#deleted those cols columns from the dataset as they only contain nulls

Queens = Queens.drop(cols, axis=1)

In [2140]:
Queens['new_date'] = pd.to_datetime(Queens['status_date'], errors='coerce')

In [2141]:
Queens = Queens.sort_values(by= ['new_date'])

In [2142]:
Queens['new_date'].loc[(Queens['new_date'] < '2010-04-01')] = '2010-04-01'

In [2143]:
Queens = Queens.sort_values(by= ['location_code', 'period'])

In [2144]:
Queens['locaton_code'] = Queens['location_code'].astype("category")

In [2145]:
locations = Queens['location_code'].unique()

In [2146]:
Queens.loc[Queens['name'].isnull(), 'name'] = Queens['location_code']

In [2147]:
Queens['status_date'] = pd.to_datetime(Queens['status_date'], errors='coerce')

In [2148]:
Queens['status_date'].iloc[Queens['status_date'] < '2010-04-01'] = '04/01/2010'

In [2149]:
Queens['status_date'] = pd.to_datetime(Queens['status_date'], errors='coerce')

In [2150]:
Queens['period2'] = pd.to_datetime(Queens['period'])

In [2151]:
Queens['period2'] = Queens['period2'] + pd.DateOffset(months=1)

In [2152]:
Queens['duration'] = Queens['period2'] - Queens['status_date']

In [2153]:
Queens['duration'] = Queens['duration'].dt.days

In [2154]:
Queens['study-start'] = '04/01/2010'

In [2155]:
Queens['study-start'] = pd.to_datetime(Queens['study-start'], errors='coerce')

In [2156]:
Queens['time-of-entry'] = Queens['status_date'] -  Queens['study-start']

In [2157]:
Queens['time-at-open'] = 0

In [2158]:
Queens['vacant'] = 1

In [2159]:
Queens = Queens.drop(Queens[(Queens['status'] == True)].index)

In [2160]:
Queens['location_code'] = Queens['location_code'].astype("category")

In [2161]:
locations = list(Queens['location_code'].unique())

In [2162]:
Queens = Queens.set_index('location_code')

In [2163]:
Queens['survival'] = 0

In [2164]:
Queens['survival'] = Queens['duration']

In [2165]:
Queens['entry'] = Queens['time-of-entry'].dt.days

In [2166]:
Queens['time-at-open'] = Queens['survival'] + Queens['entry']

In [2167]:
Queens['stat'] = Queens['status'].astype(int)

In [2168]:
Queens["StatusChange"] =  Queens['stat'].shift() != Queens['stat']

In [2169]:
Queens["StatusChange"].iloc[0] = False

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [2170]:
Queens["TenantChange"] =  Queens['name'].shift(-1) != Queens['name'] 

In [2171]:
Queens["TenantChange"].iloc[0] = False

In [2172]:
counter = 0 
stoppingCondition = len(locations)
Queens2 = pd.DataFrame(data=None, columns=Queens.columns, index=None)

while counter < stoppingCondition:
        i = locations[counter]
        m = Queens.loc[i, :].iloc[[ Queens.loc[i, :]['status_duration'].argmax()]]
        Queens2 = Queens2.append(m, ignore_index=False)
        counter += 1

In [2173]:
Queens3 = Queens2.drop_duplicates()

In [2174]:
Queens3.loc[Queens3['time-at-open'] == 4018, ['vacant']] = 0

In [2175]:
Queens3['highstreet'] = 'Queens'

In [2176]:
Queens3.to_csv('Queens-vacant-survival.csv')

#### 12. Kilburn

In [2177]:
Kilburn = Kilburn.sort_values(by= ['status_date'])

In [2178]:
cols = ['V3', 'V4', 'V5', 'V13', 'V14', 'V15', 'V16', 'V17', 'V19', 'V20', 'V21', 'V26', 'V28', 'V30', 'V36', 'V38', 'V39', 'V48', 'V51', 'V54', 'V55', 'V56', 'V57', 
        'V58', 'V59', 'V60', 'V64', 'V70', 'V72', 'V77']

In [2179]:
#deleted those cols columns from the dataset as they only contain nulls

Kilburn = Kilburn.drop(cols, axis=1)

In [2180]:
Kilburn['new_date'] = pd.to_datetime(Kilburn['status_date'], errors='coerce')

In [2181]:
Kilburn = Kilburn.sort_values(by= ['new_date'])

In [2182]:
Kilburn['new_date'].loc[(Kilburn['new_date'] < '2010-04-01')] = '2010-04-01'

In [2183]:
Kilburn = Kilburn.sort_values(by= ['location_code', 'period'])

In [2184]:
Kilburn['locaton_code'] = Kilburn['location_code'].astype("category")

In [2185]:
locations = Kilburn['location_code'].unique()

In [2186]:
Kilburn.loc[Kilburn['name'].isnull(), 'name'] = Kilburn['location_code']

In [2187]:
Kilburn['status_date'] = pd.to_datetime(Kilburn['status_date'], errors='coerce')

In [2188]:
Kilburn['status_date'].iloc[Kilburn['status_date'] < '2010-04-01'] = '04/01/2010'

In [2189]:
Kilburn['status_date'] = pd.to_datetime(Kilburn['status_date'], errors='coerce')

In [2190]:
Kilburn['period2'] = pd.to_datetime(Kilburn['period'])

In [2191]:
Kilburn['period2'] = Kilburn['period2'] + pd.DateOffset(months=1)

In [2192]:
Kilburn['duration'] = Kilburn['period2'] - Kilburn['status_date']

In [2193]:
Kilburn['duration'] = Kilburn['duration'].dt.days

In [2194]:
Kilburn['study-start'] = '04/01/2010'

In [2195]:
Kilburn['study-start'] = pd.to_datetime(Kilburn['study-start'], errors='coerce')

In [2196]:
Kilburn['time-of-entry'] = Kilburn['status_date'] -  Kilburn['study-start']

In [2197]:
Kilburn['time-at-open'] = 0

In [2198]:
Kilburn['vacant'] = 1

In [2199]:
Kilburn = Kilburn.drop(Kilburn[(Kilburn['status'] == True)].index)

In [2200]:
Kilburn['location_code'] = Kilburn['location_code'].astype("category")

In [2201]:
locations = list(Kilburn['location_code'].unique())

In [2202]:
Kilburn = Kilburn.set_index('location_code')

In [2203]:
Kilburn['survival'] = 0

In [2204]:
Kilburn['survival'] = Kilburn['duration']

In [2205]:
Kilburn['entry'] = Kilburn['time-of-entry'].dt.days

In [2206]:
Kilburn['time-at-open'] = Kilburn['survival'] + Kilburn['entry']

In [2207]:
Kilburn['stat'] = Kilburn['status'].astype(int)

In [2208]:
Kilburn["StatusChange"] =  Kilburn['stat'].shift() != Kilburn['stat']

In [2209]:
Kilburn["StatusChange"].iloc[0] = False

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [2210]:
Kilburn["TenantChange"] =  Kilburn['name'].shift(-1) != Kilburn['name'] 

In [2211]:
Kilburn["TenantChange"].iloc[0] = False

In [2212]:
counter = 0 
stoppingCondition = len(locations)
Kilburn2 = pd.DataFrame(data=None, columns=Kilburn.columns, index=None)

while counter < stoppingCondition:
        i = locations[counter]
        m = Kilburn.loc[i, :].iloc[[ Kilburn.loc[i, :]['status_duration'].argmax()]]
        Kilburn2 = Kilburn2.append(m, ignore_index=False)
        counter += 1

In [2213]:
Kilburn3 = Kilburn2.drop_duplicates()

In [2214]:
Kilburn3.loc[Kilburn3['time-at-open'] == 4018, ['vacant']] = 0

In [2215]:
Kilburn3['highstreet'] = 'Kilburn'

In [2216]:
Kilburn3.to_csv('Kilburn-vacant-survival.csv')

#### 13. Mill

In [2217]:
Mill = Mill.sort_values(by= ['status_date'])

In [2218]:
cols = ['V3', 'V4', 'V5', 'V13', 'V14', 'V15', 'V16', 'V17', 'V19', 'V20', 'V21', 'V26', 'V28', 'V30', 'V36', 'V38', 'V39', 'V48', 'V51', 'V54', 'V55', 'V56', 'V57', 
        'V58', 'V59', 'V60', 'V64', 'V70', 'V72', 'V77']

In [2219]:
#deleted those cols columns from the dataset as they only contain nulls

Mill = Mill.drop(cols, axis=1)

In [2220]:
Mill['new_date'] = pd.to_datetime(Mill['status_date'], errors='coerce')

In [2221]:
Mill = Mill.sort_values(by= ['new_date'])

In [2222]:
Mill['new_date'].loc[(Mill['new_date'] < '2010-04-01')] = '2010-04-01'

In [2223]:
Mill = Mill.sort_values(by= ['location_code', 'period'])

In [2224]:
Mill['locaton_code'] = Mill['location_code'].astype("category")

In [2225]:
locations = Mill['location_code'].unique()

In [2226]:
Mill.loc[Mill['name'].isnull(), 'name'] = Mill['location_code']

In [2227]:
Mill['status_date'] = pd.to_datetime(Mill['status_date'], errors='coerce')

In [2228]:
Mill['status_date'].iloc[Mill['status_date'] < '2010-04-01'] = '04/01/2010'

In [2229]:
Mill['status_date'] = pd.to_datetime(Mill['status_date'], errors='coerce')

In [2230]:
Mill['period2'] = pd.to_datetime(Mill['period'])

In [2231]:
Mill['period2'] = Mill['period2'] + pd.DateOffset(months=1)

In [2232]:
Mill['duration'] = Mill['period2'] - Mill['status_date']

In [2233]:
Mill['duration'] = Mill['duration'].dt.days

In [2234]:
Mill['study-start'] = '04/01/2010'

In [2235]:
Mill['study-start'] = pd.to_datetime(Mill['study-start'], errors='coerce')

In [2236]:
Mill['time-of-entry'] = Mill['status_date'] -  Mill['study-start']

In [2237]:
Mill['time-at-open'] = 0

In [2238]:
Mill['vacant'] = 1

In [2239]:
Mill = Mill.drop(Mill[(Mill['status'] == True)].index)

In [2240]:
Mill['location_code'] = Mill['location_code'].astype("category")

In [2241]:
locations = list(Mill['location_code'].unique())

In [2242]:
Mill = Mill.set_index('location_code')

In [2243]:
Mill['survival'] = 0

In [2244]:
Mill['survival'] = Mill['duration']

In [2245]:
Mill['entry'] = Mill['time-of-entry'].dt.days

In [2246]:
Mill['time-at-open'] = Mill['survival'] + Mill['entry']

In [2247]:
Mill['stat'] = Mill['status'].astype(int)

In [2248]:
Mill["StatusChange"] =  Mill['stat'].shift() != Mill['stat']

In [2249]:
Mill["StatusChange"].iloc[0] = False

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [2250]:
Mill["TenantChange"] =  Mill['name'].shift(-1) != Mill['name'] 

In [2251]:
Mill["TenantChange"].iloc[0] = False

In [2252]:
counter = 0 
stoppingCondition = len(locations)
Mill2 = pd.DataFrame(data=None, columns=Mill.columns, index=None)

while counter < stoppingCondition:
        i = locations[counter]
        m = Mill.loc[i, :].iloc[[ Mill.loc[i, :]['status_duration'].argmax()]]
        Mill2 = Mill2.append(m, ignore_index=False)
        counter += 1

In [2253]:
Mill3 = Mill2.drop_duplicates()

In [2254]:
Mill3.loc[Mill3['time-at-open'] == 4018, ['vacant']] = 0

In [2255]:
Mill3['highstreet'] = 'Mill'

In [2256]:
Mill3.to_csv('Mill-vacant-survival.csv')

#### 14. Fortune

In [2257]:
Fortune = Fortune.sort_values(by= ['status_date'])

In [2258]:
cols = ['V3', 'V4', 'V5', 'V13', 'V14', 'V15', 'V16', 'V17', 'V19', 'V20', 'V21', 'V26', 'V28', 'V30', 'V36', 'V38', 'V39', 'V48', 'V51', 'V54', 'V55', 'V56', 'V57', 
        'V58', 'V59', 'V60', 'V64', 'V70', 'V72', 'V77']

In [2259]:
#deleted those cols columns from the dataset as they only contain nulls

Fortune = Fortune.drop(cols, axis=1)

In [2260]:
Fortune['new_date'] = pd.to_datetime(Fortune['status_date'], errors='coerce')

In [2261]:
Fortune = Fortune.sort_values(by= ['new_date'])

In [2262]:
Fortune['new_date'].loc[(Fortune['new_date'] < '2010-04-01')] = '2010-04-01'

In [2263]:
Fortune = Fortune.sort_values(by= ['location_code', 'period'])

In [2264]:
Fortune['locaton_code'] = Fortune['location_code'].astype("category")

In [2265]:
locations = Fortune['location_code'].unique()

In [2266]:
Fortune.loc[Fortune['name'].isnull(), 'name'] = Fortune['location_code']

In [2267]:
Fortune['status_date'] = pd.to_datetime(Fortune['status_date'], errors='coerce')

In [2268]:
Fortune['status_date'].iloc[Fortune['status_date'] < '2010-04-01'] = '04/01/2010'

In [2269]:
Fortune['status_date'] = pd.to_datetime(Fortune['status_date'], errors='coerce')

In [2270]:
Fortune['period2'] = pd.to_datetime(Fortune['period'])

In [2271]:
Fortune['period2'] = Fortune['period2'] + pd.DateOffset(months=1)

In [2272]:
Fortune['duration'] = Fortune['period2'] - Fortune['status_date']

In [2273]:
Fortune['duration'] = Fortune['duration'].dt.days

In [2274]:
Fortune['study-start'] = '04/01/2010'

In [2275]:
Fortune['study-start'] = pd.to_datetime(Fortune['study-start'], errors='coerce')

In [2276]:
Fortune['time-of-entry'] = Fortune['status_date'] -  Fortune['study-start']

In [2277]:
Fortune['time-at-open'] = 0

In [2278]:
Fortune['vacant'] = 1

In [2279]:
Fortune = Fortune.drop(Fortune[(Fortune['status'] == True)].index)

In [2280]:
Fortune['location_code'] = Fortune['location_code'].astype("category")

In [2281]:
locations = list(Fortune['location_code'].unique())

In [2282]:
Fortune = Fortune.set_index('location_code')

In [2283]:
Fortune['survival'] = 0

In [2284]:
Fortune['survival'] = Fortune['duration']

In [2285]:
Fortune['entry'] = Fortune['time-of-entry'].dt.days

In [2286]:
Fortune['time-at-open'] = Fortune['survival'] + Fortune['entry']

In [2287]:
Fortune['stat'] = Fortune['status'].astype(int)

In [2288]:
Fortune["StatusChange"] =  Fortune['stat'].shift() != Fortune['stat']

In [2289]:
Fortune["StatusChange"].iloc[0] = False

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [2290]:
Fortune["TenantChange"] =  Fortune['name'].shift(-1) != Fortune['name'] 

In [2291]:
Fortune["TenantChange"].iloc[0] = False

In [2292]:
counter = 0 
stoppingCondition = len(locations)
Fortune2 = pd.DataFrame(data=None, columns=Fortune.columns, index=None)

while counter < stoppingCondition:
        i = locations[counter]
        m = Fortune.loc[i, :].iloc[[ Fortune.loc[i, :]['status_duration'].argmax()]]
        Fortune2 = Fortune2.append(m, ignore_index=False)
        counter += 1

In [2293]:
Fortune3 = Fortune2.drop_duplicates()

In [2294]:
Fortune3.loc[Fortune3['time-at-open'] == 4018, ['vacant']] = 0

In [2295]:
Fortune3['highstreet'] = 'Fortune'

In [2296]:
Fortune3.to_csv('Fortune-vacant-survival.csv')

#### 15. Willesden

In [2297]:
Willesden = Willesden.sort_values(by= ['status_date'])

In [2298]:
cols = ['V3', 'V4', 'V5', 'V13', 'V14', 'V15', 'V16', 'V17', 'V19', 'V20', 'V21', 'V26', 'V28', 'V30', 'V36', 'V38', 'V39', 'V48', 'V51', 'V54', 'V55', 'V56', 'V57', 
        'V58', 'V59', 'V60', 'V64', 'V70', 'V72', 'V77']

In [2299]:
#deleted those cols columns from the dataset as they only contain nulls

Willesden = Willesden.drop(cols, axis=1)

In [2300]:
Willesden['new_date'] = pd.to_datetime(Willesden['status_date'], errors='coerce')

In [2301]:
Willesden = Willesden.sort_values(by= ['new_date'])

In [2302]:
Willesden['new_date'].loc[(Willesden['new_date'] < '2010-04-01')] = '2010-04-01'

In [2303]:
Willesden = Willesden.sort_values(by= ['location_code', 'period'])

In [2304]:
Willesden['locaton_code'] = Willesden['location_code'].astype("category")

In [2305]:
locations = Willesden['location_code'].unique()

In [2306]:
Willesden.loc[Willesden['name'].isnull(), 'name'] = Willesden['location_code']

In [2307]:
Willesden['status_date'] = pd.to_datetime(Willesden['status_date'], errors='coerce')

In [2308]:
Willesden['status_date'].iloc[Willesden['status_date'] < '2010-04-01'] = '04/01/2010'

In [2309]:
Willesden['status_date'] = pd.to_datetime(Willesden['status_date'], errors='coerce')

In [2310]:
Willesden['period2'] = pd.to_datetime(Willesden['period'])

In [2311]:
Willesden['period2'] = Willesden['period2'] + pd.DateOffset(months=1)

In [2312]:
Willesden['duration'] = Willesden['period2'] - Willesden['status_date']

In [2313]:
Willesden['duration'] = Willesden['duration'].dt.days

In [2314]:
Willesden['study-start'] = '04/01/2010'

In [2315]:
Willesden['study-start'] = pd.to_datetime(Willesden['study-start'], errors='coerce')

In [2316]:
Willesden['time-of-entry'] = Willesden['status_date'] -  Willesden['study-start']

In [2317]:
Willesden['time-at-open'] = 0

In [2318]:
Willesden['vacant'] = 1

In [2319]:
Willesden = Willesden.drop(Willesden[(Willesden['status'] == True)].index)

In [2320]:
Willesden['location_code'] = Willesden['location_code'].astype("category")

In [2321]:
locations = list(Willesden['location_code'].unique())

In [2322]:
Willesden = Willesden.set_index('location_code')

In [2323]:
Willesden['survival'] = 0

In [2324]:
Willesden['survival'] = Willesden['duration']

In [2325]:
Willesden['entry'] = Willesden['time-of-entry'].dt.days

In [2326]:
Willesden['time-at-open'] = Willesden['survival'] + Willesden['entry']

In [2327]:
Willesden['stat'] = Willesden['status'].astype(int)

In [2328]:
Willesden["StatusChange"] =  Willesden['stat'].shift() != Willesden['stat']

In [2329]:
Willesden["StatusChange"].iloc[0] = False

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [2330]:
Willesden["TenantChange"] =  Willesden['name'].shift(-1) != Willesden['name'] 

In [2331]:
Willesden["TenantChange"].iloc[0] = False

In [2332]:
counter = 0 
stoppingCondition = len(locations)
Willesden2 = pd.DataFrame(data=None, columns=Willesden.columns, index=None)

while counter < stoppingCondition:
        i = locations[counter]
        m = Willesden.loc[i, :].iloc[[ Willesden.loc[i, :]['status_duration'].argmax()]]
        Willesden2 = Willesden2.append(m, ignore_index=False)
        counter += 1

In [2333]:
Willesden3 = Willesden2.drop_duplicates()

In [2334]:
Willesden3.loc[Willesden3['time-at-open'] == 4018, ['vacant']] = 0

In [2335]:
Willesden3['highstreet'] = 'Willesden'

In [2336]:
Willesden3.to_csv('Willesden-vacant-survival.csv')

#### 16. Mansfield

In [2337]:
Mansfield = Mansfield.sort_values(by= ['status_date'])

In [2338]:
cols = ['V3', 'V4', 'V5', 'V13', 'V14', 'V15', 'V16', 'V17', 'V19', 'V20', 'V21', 'V26', 'V28', 'V30', 'V36', 'V38', 'V39', 'V48', 'V51', 'V54', 'V55', 'V56', 'V57', 
        'V58', 'V59', 'V60', 'V64', 'V70', 'V72', 'V77']

In [2339]:
#deleted those cols columns from the dataset as they only contain nulls

Mansfield = Mansfield.drop(cols, axis=1)

In [2340]:
Mansfield['new_date'] = pd.to_datetime(Mansfield['status_date'], errors='coerce')

In [2341]:
Mansfield = Mansfield.sort_values(by= ['new_date'])

In [2342]:
Mansfield['new_date'].loc[(Mansfield['new_date'] < '2010-04-01')] = '2010-04-01'

In [2343]:
Mansfield = Mansfield.sort_values(by= ['location_code', 'period'])

In [2344]:
Mansfield['locaton_code'] = Mansfield['location_code'].astype("category")

In [2345]:
locations = Mansfield['location_code'].unique()

In [2346]:
Mansfield.loc[Mansfield['name'].isnull(), 'name'] = Mansfield['location_code']

In [2347]:
Mansfield['status_date'] = pd.to_datetime(Mansfield['status_date'], errors='coerce')

In [2348]:
Mansfield['status_date'].iloc[Mansfield['status_date'] < '2010-04-01'] = '04/01/2010'

In [2349]:
Mansfield['status_date'] = pd.to_datetime(Mansfield['status_date'], errors='coerce')

In [2350]:
Mansfield['period2'] = pd.to_datetime(Mansfield['period'])

In [2351]:
Mansfield['period2'] = Mansfield['period2'] + pd.DateOffset(months=1)

In [2352]:
Mansfield['duration'] = Mansfield['period2'] - Mansfield['status_date']

In [2353]:
Mansfield['duration'] = Mansfield['duration'].dt.days

In [2354]:
Mansfield['study-start'] = '04/01/2010'

In [2355]:
Mansfield['study-start'] = pd.to_datetime(Mansfield['study-start'], errors='coerce')

In [2356]:
Mansfield['time-of-entry'] = Mansfield['status_date'] -  Mansfield['study-start']

In [2357]:
Mansfield['time-at-open'] = 0

In [2358]:
Mansfield['vacant'] = 1

In [2359]:
Mansfield = Mansfield.drop(Mansfield[(Mansfield['status'] == True)].index)

In [2360]:
Mansfield['location_code'] = Mansfield['location_code'].astype("category")

In [2361]:
locations = list(Mansfield['location_code'].unique())

In [2362]:
Mansfield = Mansfield.set_index('location_code')

In [2363]:
Mansfield['survival'] = 0

In [2364]:
Mansfield['survival'] = Mansfield['duration']

In [2365]:
Mansfield['entry'] = Mansfield['time-of-entry'].dt.days

In [2366]:
Mansfield['time-at-open'] = Mansfield['survival'] + Mansfield['entry']

In [2367]:
Mansfield['stat'] = Mansfield['status'].astype(int)

In [2368]:
Mansfield["StatusChange"] =  Mansfield['stat'].shift() != Mansfield['stat']

In [2369]:
Mansfield["StatusChange"].iloc[0] = False

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [2370]:
Mansfield["TenantChange"] =  Mansfield['name'].shift(-1) != Mansfield['name'] 

In [2371]:
Mansfield["TenantChange"].iloc[0] = False

In [2499]:
Mansfield

,Unnamed: 0,Unnamed: 0.1,rates_code,rates_authority_id,rates_authority,region,address_no,address_street,address_town,pc__pcs,...,duration,study-start,time-of-entry,time-at-open,vacant,survival,entry,stat,StatusChange,TenantChange
location_code,,,,,,,,,,,,,,,,,,,,,
58763063,29480,29480,00210006810015,E09000007,Camden,Inner London - West,68,MANSFIELD ROAD,LONDON,NW3 2HU,...,91,2010-04-01,0 days,91,1,91,0,0,False,False
58763063,29481,29481,00210006810015,E09000007,Camden,Inner London - West,68,MANSFIELD ROAD,LONDON,NW3 2HU,...,183,2010-04-01,0 days,183,1,183,0,0,False,False
58763063,29482,29482,00210006810015,E09000007,Camden,Inner London - West,68,MANSFIELD ROAD,LONDON,NW3 2HU,...,275,2010-04-01,0 days,275,1,275,0,0,False,False
58763063,29483,29483,00210006810015,E09000007,Camden,Inner London - West,68,MANSFIELD ROAD,LONDON,NW3 2HU,...,365,2010-04-01,0 days,365,1,365,0,0,False,False
58763063,29484,29484,00210006810015,E09000007,Camden,Inner London - West,68,MANSFIELD ROAD,LONDON,NW3 2HU,...,456,2010-04-01,0 days,456,1,456,0,0,False,False
58763063,29485,29485,2.10007E+11,E09000007,Camden,Inner London - West,68,MANSFIELD ROAD,LONDON,NW3 2HU,...,548,2010-04-01,0 days,548,1,548,0,0,False,False
58763063,29486,29486,00210006810015,E09000007,Camden,Inner London - West,68,MANSFIELD ROAD,LONDON,NW3 2HU,...,640,2010-04-01,0 days,640,1,640,0,0,False,False
58763063,29487,29487,00210006810015,E09000007,Camden,Inner London - West,68,MANSFIELD ROAD,LONDON,NW3 2HU,...,731,2010-04-01,0 days,731,1,731,0,0,False,False
58763063,29488,29488,00210006810015,E09000007,Camden,Inner London - West,68,MANSFIELD ROAD,LONDON,NW3 2HU,...,822,2010-04-01,0 days,822,1,822,0,0,False,False


In [2372]:
counter = 0 
stoppingCondition = len(locations)
Mansfield2 = pd.DataFrame(data=None, columns=Mansfield.columns, index=None)

while counter < stoppingCondition:
        i = locations[counter]
        m = Mansfield.loc[i, :].iloc[[ Mansfield.loc[i, :]['status_duration'].argmax()]]
        Mansfield2 = Mansfield2.append(m, ignore_index=False)
        counter += 1

In [2373]:
Mansfield3 = Mansfield2.drop_duplicates()

In [2374]:
Mansfield3.loc[Mansfield3['time-at-open'] == 4018, ['vacant']] = 0

In [2375]:
Mansfield3['highstreet'] = 'Mansfield'

In [2376]:
Mansfield3.to_csv('Mansfield-vacant-survival.csv')

#### 17. WestEnd

In [2377]:
WestEnd = WestEnd.sort_values(by= ['status_date'])

In [2378]:
cols = ['V3', 'V4', 'V5', 'V13', 'V14', 'V15', 'V16', 'V17', 'V19', 'V20', 'V21', 'V26', 'V28', 'V30', 'V36', 'V38', 'V39', 'V48', 'V51', 'V54', 'V55', 'V56', 'V57', 
        'V58', 'V59', 'V60', 'V64', 'V70', 'V72', 'V77']

In [2379]:
#deleted those cols columns from the dataset as they only contain nulls

WestEnd = WestEnd.drop(cols, axis=1)

In [2380]:
WestEnd['new_date'] = pd.to_datetime(WestEnd['status_date'], errors='coerce')

In [2381]:
WestEnd = WestEnd.sort_values(by= ['new_date'])

In [2382]:
WestEnd['new_date'].loc[(WestEnd['new_date'] < '2010-04-01')] = '2010-04-01'

In [2383]:
WestEnd = WestEnd.sort_values(by= ['location_code', 'period'])

In [2384]:
WestEnd['locaton_code'] = WestEnd['location_code'].astype("category")

In [2385]:
locations = WestEnd['location_code'].unique()

In [2386]:
WestEnd.loc[WestEnd['name'].isnull(), 'name'] = WestEnd['location_code']

In [2387]:
WestEnd['status_date'] = pd.to_datetime(WestEnd['status_date'], errors='coerce')

In [2388]:
WestEnd['status_date'].iloc[WestEnd['status_date'] < '2010-04-01'] = '04/01/2010'

In [2389]:
WestEnd['status_date'] = pd.to_datetime(WestEnd['status_date'], errors='coerce')

In [2390]:
WestEnd['period2'] = pd.to_datetime(WestEnd['period'])

In [2391]:
WestEnd['period2'] = WestEnd['period2'] + pd.DateOffset(months=1)

In [2392]:
WestEnd['duration'] = WestEnd['period2'] - WestEnd['status_date']

In [2393]:
WestEnd['duration'] = WestEnd['duration'].dt.days

In [2394]:
WestEnd['study-start'] = '04/01/2010'

In [2395]:
WestEnd['study-start'] = pd.to_datetime(WestEnd['study-start'], errors='coerce')

In [2396]:
WestEnd['time-of-entry'] = WestEnd['status_date'] -  WestEnd['study-start']

In [2397]:
WestEnd['time-at-open'] = 0

In [2398]:
WestEnd['vacant'] = 1

In [2399]:
WestEnd = WestEnd.drop(WestEnd[(WestEnd['status'] == True)].index)

In [2400]:
WestEnd['location_code'] = WestEnd['location_code'].astype("category")

In [2401]:
locations = list(WestEnd['location_code'].unique())

In [2402]:
WestEnd = WestEnd.set_index('location_code')

In [2403]:
WestEnd['survival'] = 0

In [2404]:
WestEnd['survival'] = WestEnd['duration']

In [2405]:
WestEnd['entry'] = WestEnd['time-of-entry'].dt.days

In [2406]:
WestEnd['time-at-open'] = WestEnd['survival'] + WestEnd['entry']

In [2407]:
WestEnd['stat'] = WestEnd['status'].astype(int)

In [2408]:
WestEnd["StatusChange"] =  WestEnd['stat'].shift() != WestEnd['stat']

In [2409]:
WestEnd["StatusChange"].iloc[0] = False

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [2410]:
WestEnd["TenantChange"] =  WestEnd['name'].shift(-1) != WestEnd['name'] 

In [2411]:
WestEnd["TenantChange"].iloc[0] = False

In [2412]:
counter = 0 
stoppingCondition = len(locations)
WestEnd2 = pd.DataFrame(data=None, columns=WestEnd.columns, index=None)

while counter < stoppingCondition:
        i = locations[counter]
        m = WestEnd.loc[i, :].iloc[[ WestEnd.loc[i, :]['status_duration'].argmax()]]
        WestEnd2 = WestEnd2.append(m, ignore_index=False)
        counter += 1

In [2413]:
WestEnd3 = WestEnd2.drop_duplicates()

In [2414]:
WestEnd3.loc[WestEnd3['time-at-open'] == 4018, ['vacant']] = 0

In [2415]:
WestEnd3['highstreet'] = 'WestEnd'

In [2416]:
WestEnd3.to_csv('WestEnd-vacant-survival.csv')

#### 18. Hampstead

In [2417]:
Hampstead = Hampstead.sort_values(by= ['status_date'])

In [2418]:
cols = ['V3', 'V4', 'V5', 'V13', 'V14', 'V15', 'V16', 'V17', 'V19', 'V20', 'V21', 'V26', 'V28', 'V30', 'V36', 'V38', 'V39', 'V48', 'V51', 'V54', 'V55', 'V56', 'V57', 
        'V58', 'V59', 'V60', 'V64', 'V70', 'V72', 'V77']

In [2419]:
#deleted those cols columns from the dataset as they only contain nulls

Hampstead = Hampstead.drop(cols, axis=1)

In [2420]:
Hampstead['new_date'] = pd.to_datetime(Hampstead['status_date'], errors='coerce')

In [2421]:
Hampstead = Hampstead.sort_values(by= ['new_date'])

In [2422]:
Hampstead['new_date'].loc[(Hampstead['new_date'] < '2010-04-01')] = '2010-04-01'

In [2423]:
Hampstead = Hampstead.sort_values(by= ['location_code', 'period'])

In [2424]:
Hampstead['locaton_code'] = Hampstead['location_code'].astype("category")

In [2425]:
locations = Hampstead['location_code'].unique()

In [2426]:
Hampstead.loc[Hampstead['name'].isnull(), 'name'] = Hampstead['location_code']

In [2427]:
Hampstead['status_date'] = pd.to_datetime(Hampstead['status_date'], errors='coerce')

In [2428]:
Hampstead['status_date'].iloc[Hampstead['status_date'] < '2010-04-01'] = '04/01/2010'

In [2429]:
Hampstead['status_date'] = pd.to_datetime(Hampstead['status_date'], errors='coerce')

In [2430]:
Hampstead['period2'] = pd.to_datetime(Hampstead['period'])

In [2431]:
Hampstead['period2'] = Hampstead['period2'] + pd.DateOffset(months=1)

In [2432]:
Hampstead['duration'] = Hampstead['period2'] - Hampstead['status_date']

In [2433]:
Hampstead['duration'] = Hampstead['duration'].dt.days

In [2434]:
Hampstead['study-start'] = '04/01/2010'

In [2435]:
Hampstead['study-start'] = pd.to_datetime(Hampstead['study-start'], errors='coerce')

In [2436]:
Hampstead['time-of-entry'] = Hampstead['status_date'] -  Hampstead['study-start']

In [2437]:
Hampstead['time-at-open'] = 0

In [2438]:
Hampstead['vacant'] = 1

In [2439]:
Hampstead = Hampstead.drop(Hampstead[(Hampstead['status'] == True)].index)

In [2440]:
Hampstead['location_code'] = Hampstead['location_code'].astype("category")

In [2441]:
locations = list(Hampstead['location_code'].unique())

In [2442]:
Hampstead = Hampstead.set_index('location_code')

In [2443]:
Hampstead['survival'] = 0

In [2444]:
Hampstead['survival'] = Hampstead['duration']

In [2445]:
Hampstead['entry'] = Hampstead['time-of-entry'].dt.days

In [2446]:
Hampstead['time-at-open'] = Hampstead['survival'] + Hampstead['entry']

In [2447]:
Hampstead['stat'] = Hampstead['status'].astype(int)

In [2448]:
Hampstead["StatusChange"] =  Hampstead['stat'].shift() != Hampstead['stat']

In [2449]:
Hampstead["StatusChange"].iloc[0] = False

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [2450]:
Hampstead["TenantChange"] =  Hampstead['name'].shift(-1) != Hampstead['name'] 

In [2451]:
Hampstead["TenantChange"].iloc[0] = False

In [2452]:
counter = 0 
stoppingCondition = len(locations)
Hampstead2 = pd.DataFrame(data=None, columns=Hampstead.columns, index=None)

while counter < stoppingCondition:
        i = locations[counter]
        m = Hampstead.loc[i, :].iloc[[ Hampstead.loc[i, :]['status_duration'].argmax()]]
        Hampstead2 = Hampstead2.append(m, ignore_index=False)
        counter += 1

In [2453]:
Hampstead3 = Hampstead2.drop_duplicates()

In [2454]:
Hampstead3.loc[Hampstead3['time-at-open'] == 4018, ['vacant']] = 0

In [2455]:
Hampstead3['highstreet'] = 'Hampstead'

In [2456]:
Hampstead3.to_csv('Hampstead-vacant-survival.csv')

#### 19. Eversholt

In [2457]:
Eversholt = Eversholt.sort_values(by= ['status_date'])

In [2458]:
cols = ['V3', 'V4', 'V5', 'V13', 'V14', 'V15', 'V16', 'V17', 'V19', 'V20', 'V21', 'V26', 'V28', 'V30', 'V36', 'V38', 'V39', 'V48', 'V51', 'V54', 'V55', 'V56', 'V57', 
        'V58', 'V59', 'V60', 'V64', 'V70', 'V72', 'V77']

In [2459]:
#deleted those cols columns from the dataset as they only contain nulls

Eversholt = Eversholt.drop(cols, axis=1)

In [2460]:
Eversholt['new_date'] = pd.to_datetime(Eversholt['status_date'], errors='coerce')

In [2461]:
Eversholt = Eversholt.sort_values(by= ['new_date'])

In [2462]:
Eversholt['new_date'].loc[(Eversholt['new_date'] < '2010-04-01')] = '2010-04-01'

In [2463]:
Eversholt = Eversholt.sort_values(by= ['location_code', 'period'])

In [2464]:
Eversholt['locaton_code'] = Eversholt['location_code'].astype("category")

In [2465]:
locations = Eversholt['location_code'].unique()

In [2466]:
Eversholt.loc[Eversholt['name'].isnull(), 'name'] = Eversholt['location_code']

In [2467]:
Eversholt['status_date'] = pd.to_datetime(Eversholt['status_date'], errors='coerce')

In [2468]:
Eversholt['status_date'].iloc[Eversholt['status_date'] < '2010-04-01'] = '04/01/2010'

In [2469]:
Eversholt['status_date'] = pd.to_datetime(Eversholt['status_date'], errors='coerce')

In [2470]:
Eversholt['period2'] = pd.to_datetime(Eversholt['period'])

In [2471]:
Eversholt['period2'] = Eversholt['period2'] + pd.DateOffset(months=1)

In [2472]:
Eversholt['duration'] = Eversholt['period2'] - Eversholt['status_date']

In [2473]:
Eversholt['duration'] = Eversholt['duration'].dt.days

In [2474]:
Eversholt['study-start'] = '04/01/2010'

In [2475]:
Eversholt['study-start'] = pd.to_datetime(Eversholt['study-start'], errors='coerce')

In [2476]:
Eversholt['time-of-entry'] = Eversholt['status_date'] -  Eversholt['study-start']

In [2477]:
Eversholt['time-at-open'] = 0

In [2478]:
Eversholt['vacant'] = 1

In [2479]:
Eversholt = Eversholt.drop(Eversholt[(Eversholt['status'] == True)].index)

In [2480]:
Eversholt['location_code'] = Eversholt['location_code'].astype("category")

In [2481]:
locations = list(Eversholt['location_code'].unique())

In [2482]:
Eversholt = Eversholt.set_index('location_code')

In [2483]:
Eversholt['survival'] = 0

In [2484]:
Eversholt['survival'] = Eversholt['duration']

In [2485]:
Eversholt['entry'] = Eversholt['time-of-entry'].dt.days

In [2486]:
Eversholt['time-at-open'] = Eversholt['survival'] + Eversholt['entry']

In [2487]:
Eversholt['stat'] = Eversholt['status'].astype(int)

In [2488]:
Eversholt["StatusChange"] =  Eversholt['stat'].shift() != Eversholt['stat']

In [2489]:
Eversholt["StatusChange"].iloc[0] = False

/opt/conda/envs/sds2020/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [2490]:
Eversholt["TenantChange"] =  Eversholt['name'].shift(-1) != Eversholt['name'] 

In [2491]:
Eversholt["TenantChange"].iloc[0] = False

In [2492]:
counter = 0 
stoppingCondition = len(locations)
Eversholt2 = pd.DataFrame(data=None, columns=Eversholt.columns, index=None)

while counter < stoppingCondition:
        i = locations[counter]
        m = Eversholt.loc[i, :].iloc[[ Eversholt.loc[i, :]['status_duration'].argmax()]]
        Eversholt2 = Eversholt2.append(m, ignore_index=False)
        counter += 1

In [2493]:
Eversholt3 = Eversholt2.drop_duplicates()

In [2494]:
Eversholt3.loc[Eversholt3['time-at-open'] == 4018, ['vacant']] = 0

In [2495]:
Eversholt3['highstreet'] = 'Eversholt'

In [2496]:
Eversholt3.to_csv('Eversholt-vacant-survival.csv')

****

#### Append all for Camden total survival

In [2497]:
camden_survival = pd.concat([Brecknock3, Kentish3, Fortress3, CamdenT3, Belsize3, Loudoun3, Haverstock3, SouthEnd3, Cricklewood3, Regents3, Queens3, Kilburn3, Mill3, Fortune3, Willesden3, Mansfield3, WestEnd3, Hampstead3, Eversholt3], ignore_index=True)

In [2498]:
camden_survival.to_csv('camden-vacant-survival-full.csv')